In [1]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\mini_\\Desktop\Practica-Semi2\\archivos\\Covid19.csv")
rddfiltro = text_file.rdd.map(tuple)

In [3]:
#se toma el mes y sus numeros de casos
rddCOVID= rddfiltro.map(lambda word: (word[2],word[5]))
rddCOVID.take(10)

[(9, 0),
 (9, 125),
 (9, 47),
 (9, 0),
 (9, 17),
 (9, 40),
 (9, 99),
 (9, 75),
 (9, 35),
 (9, 34)]

In [4]:
rddCONT=rddCOVID.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONT.collect())

Conteo total -> [(9, 2374517), (8, 3584040), (7, 3288723), (6, 2527885), (5, 1555025), (3, 324799), (4, 868927), (2, 73635), (1, 9758)]


In [5]:
#Top 3 meses con mayor número de pasajeros
rddORDEN= sc.parallelize(rddCONT.sortBy(lambda a: a[1],True).take(5))
print("5 meses con menos casos de COVID %s"%rddORDEN.collect())

5 meses con menos casos de COVID [(1, 9758), (2, 73635), (3, 324799), (4, 868927), (5, 1555025)]


In [6]:
#Separando nombres y totales
#Vista de datos
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

[1, 2, 3, 4, 5]
[9758, 73635, 324799, 868927, 1555025]


In [7]:
#Graficando pie
fig = go.Figure(data=go.Pie(labels=rddNombres.collect(),values=rddTotales.collect()))
fig.update_layout(title_text='5 meses con menos casos de COVID.',title_font_size=30)
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20)
fig.write_html('R3_A2.html', auto_open=True)